In [1]:
#ALL IMPORTS AND FUNCTIONS

from bs4 import BeautifulSoup
from pyvirtualdisplay import Display
import requests
import time
from datetime import datetime
import pandas as pd
import re
from requests import Session
from requests.exceptions import MissingSchema, InvalidURL, SSLError, RequestException, Timeout
import numpy as np

# GET request options
_HEADERS = {'User-Agent': 'Mozilla/5.0'}

#PLEASE UPDATE THIS VARIABLE - WE ARE UNABLE TO DYNAMICALLY SCRAPE LINK TO LATEST RANKINGS RPI PAGE FROM STATS.NCAA.COM THIS SHOULD BE UPDATED MANUALLY TO LATEST AND GET ALL LATESTS D1 NCAA SCHOOLS AND THEIR UNIQUE TEAM WEBPAGES
all_ncaa_rankings_url = 'https://stats.ncaa.org/selection_rankings/nitty_gritties/36649'

In [14]:
# FROM NCAA STATS WEBSITE (RPI PAGE OR VIA SOME BACKEND LOGIC THAT NCAA BASEBALL PACKAGE USES)
# def get_rpi_rankings_table():

with Session() as s:
    r = s.get(all_ncaa_rankings_url, headers=_HEADERS)
soup = BeautifulSoup(r.text, features='lxml')
base_url = 'https://stats.ncaa.org'
team_names = []
urls = []
school_websites = []

a_tags = soup.find_all('a')
for a in a_tags:
    href = a['href']
    # Check if the href contains "teams/[insert number]" and does not contain "team_sheet"
    if re.search(r'teams/\d+', href) and 'team_sheet' not in href:
        # Append team name and URL to the lists
        team_names.append(a.text.strip())
        team_url = base_url + href
        urls.append(team_url)

        # Get school website
        with Session() as s:
            r = s.get(team_url, headers=_HEADERS)
        soup = BeautifulSoup(r.text, features='lxml')
        a_tag = soup.find('a', {'target': 'ATHLETICS_URL'})
        school_website = a_tag['href'] if a_tag else ''
        school_websites.append(school_website)

# Create DataFrame
df = pd.DataFrame({'school': team_names, 'ncaa_team_url': urls, 'school_website_url': school_websites})

print(df)

              school                        ncaa_team_url  \
0    St. Thomas (MN)  https://stats.ncaa.org/teams/573909   
1       Tarleton St.  https://stats.ncaa.org/teams/573910   
2          Utah Tech  https://stats.ncaa.org/teams/573911   
3              Akron  https://stats.ncaa.org/teams/573912   
4          Texas A&M  https://stats.ncaa.org/teams/573913   
..               ...                                  ...   
300       Lindenwood  https://stats.ncaa.org/teams/574230   
301      Queens (NC)  https://stats.ncaa.org/teams/574231   
302    Southern Ind.  https://stats.ncaa.org/teams/574232   
303        Stonehill  https://stats.ncaa.org/teams/574233   
304         Le Moyne  https://stats.ncaa.org/teams/574412   

                      school_website_url  
0            http://www.Tommiesports.com  
1              http://tarletonsports.com  
2    http://www.utahtechtrailblazers.com  
3                      http://gozips.com  
4                 http://www.12thman.com  
..       

In [ ]:
###ADD CODE TO CLEANUP MESSY LINKS

##CONSIDER TRYING TO SCRAPE VIA LATEST RPI - SEEMS LIKE IT'S BLOCKED THOUGH

In [15]:
df.to_csv('data/all_d1_schools_and_websites.csv')